In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import load_model

In [ ]:
# Config
batch_size= 40

In [ ]:
df= pd.read_csv('../input/amz-preprocessed-csv/amz_df.csv', escapechar = "\\", quoting = 3)
df['all']= (df.TITLE+ ' ' + df.DESCRIPTION+ ' ' + df.BULLET_POINTS+ ' ' + df.BRAND)
df.shape

In [ ]:
df.head()

In [ ]:
test_csv= df[2903024:]
display(test_csv.head(3))
test_csv.shape

In [ ]:
test_csv.isnull().sum(axis = 0)

In [ ]:
test_csv= test_csv.fillna(value='error')

In [ ]:
test_csv.isnull().sum(axis = 0)

In [ ]:
def generator_test():
    samples_per_epoch = X_test.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    
    while True:
        #restart counter to yeild data in the next epoch as well
        if counter >= number_of_batches:
            counter = 0
        X= X_test[batch_size*counter:batch_size*(counter+1)]
        Y= y_test[batch_size*counter:batch_size*(counter+1)].astype('float32')
        counter += 1
        yield X,Y

In [ ]:
model = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1280, activation='relu'))
model.add(tf.keras.layers.Reshape((10, 128)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512)))
model.add(tf.keras.layers.Dropout(0.3))
model.add((tf.keras.layers.Dense(1046, activation='softmax')))
model.summary()

In [ ]:
model.load_weights('../input/fork-of-mini-model-training-amz-tittle/AMZ_model_1lstm.h5')

In [ ]:
def pred(x):
    return model.predict(x)

samples= test_csv['all'].values
samples.shape

In [ ]:
hot2label= np.load('../input/fork-of-mini-model-training-amz-tittle/1hot2label.npy')
hot2label.shape

In [ ]:
node_id=np.zeros(samples.shape[0])
for i in tqdm(range((samples.shape[0]//batch_size)+1)):
    x= samples[i*batch_size: (i+1)*batch_size]
    p= pred(x)
    node_id[i*batch_size: (i+1)*batch_size]= hot2label[np.argmax(p, -1)]

In [ ]:
sub_df= test_csv[['PRODUCT_ID', 'BROWSE_NODE_ID']]
sub_df['BROWSE_NODE_ID']= node_id
sub_df.shape

In [ ]:
sub_df.to_csv('submission.csv', index=False)

In [ ]:
sub_df.head()